In [1]:
import os, json
from letta import create_client 
from letta import EmbeddingConfig, LLMConfig
import letta

client = create_client()
client.set_default_embedding_config(EmbeddingConfig.default_config(model_name="letta"))
client.set_default_llm_config(LLMConfig.default_config(model_name="letta"))

# client.send_message(
#     agent_id="agent-2373b6c6-8fbb-4e1c-bfce-c027c3b17530",
#     message=""
# )



Initializing database...


In [ ]:
import letta.data_sources
import letta.data_sources.connectors
import json

def compareSimilarities(path):
    """
    This tool sorts the user's file system.
    If asked to sort or to organize the file system, call this function.

    Args: 
    Returns: 
    """

    file = open(path)
    fileData = json.load(file)
    #fileDataS = json.dumps(fileData)
    return fileData

file_data = compareSimilarities('FileSystemData.json')
file_keys = file_data['TestDirectory']['children'].keys()
for k in file_keys:
    print(k)

In [35]:
import os, json
from letta import create_client 
from letta import EmbeddingConfig, LLMConfig
import letta



def sort_FS(self, mode:str):
    """
    This tool sorts the user's file system.
    If asked to sort or to organize the file system, call this function.
    Tell the user the return and results of the function call.
    Query the user for the mode (valid modes are: content, extension, name, or date), before calling this function. 


    Args: 
        mode (str) : This is the mode in which the sorting should be done. Options are: content, extension, name, or date. This is chosen by the user.

    Returns: 
        fileSort (str) : This is the new organization of the file system users requested
    """
    #create another client to ask
    from letta import create_client 
    from letta import EmbeddingConfig, LLMConfig
    import json, os, re
    
    client = create_client()
    client.set_default_embedding_config(EmbeddingConfig.default_config(model_name="letta"))
    client.set_default_llm_config(LLMConfig.default_config(model_name="letta"))



    #get json data
    #file = open('FileSystemData.json') #NOTE: hardCoded
    file = open('dataTable3.json') #NOTE: hardCoded
    fileData = json.load(file)
    #fileDataS = json.dumps(fileData)

    #create message to send to different agent
    #NOTE: 1 layer simple sort, based on name
    file_keys = fileData.keys()

    
    template = "\nTemplate:\n[group 1]{item 1, item2, ...}\n[group 2]{item 3, item 4, ...}\n...\nItems:"

    enforceSingle = "If an item has already be grouped, do not group it again. Thus, for each item listed, it should only appear once amoungst all groups. Return only the output template and nothing else:"
    
    if mode == 'name':
        prompt = f"Using the template and given the following files, group the files into the template based on the commonalities in the topic, semantics, or meanings of the names. Ignore the file extension or file type"

        message = prompt + enforceSingle + template
        for k in file_keys:
            message = message + "\n" + k

    elif mode == 'extension':
        prompt = f"Using the template and given the following files, group the files into the template based on the commonalities of ONLY on the {mode}s of the file, and not on the name."

        message = prompt + enforceSingle + template
        for k in file_keys:
            message = message + "\n" + k
    elif mode  == 'content':
        #NOTE: when we say content, we are also refereing to file summary
        for k in file_keys:
            try:
                message =  message + '\n file: "' + k + '" content: "' + fileData[k]['summary'] + '"'
            except KeyError:
                continue 
    
    

    print(f'Testing Output: {message}')

    #NOTE: for easier testing, try to start fresh so we know effect of prompting
    try:
        agentState = client.create_agent('background_agent')
    except ValueError:
        agentID = client.get_agent_id('background_agent')
        client.delete_agent(agentID)
        agentState = client.create_agent('background_agent')
    
    
    answer = client.send_message(
        message=message,
        role='user',
        agent_id=agentState.id
    )

    

    answer = answer.messages[1].function_call.arguments
    answer = json.loads(answer)
    answer = json.dumps(answer['message'])
    print("MY_ANSWER", answer)

    #self.interface.assistant_message(answer)  # , msg_obj=self._messages[-1])

    outputFile = open("newFileSystem.txt", 'w')
    outputFile.write(answer)
    
    return answer

sort_FS_tool = client.create_tool(sort_FS)


In [36]:
sort_FS('_', 'extension')

Testing Output: Using the template and given the following files, group the files into the template based on the commonalities of ONLY on the extensions of the file, and not on the name.If an item has already be grouped, do not group it again. Thus, for each item listed, it should only appear once amoungst all groups. Return only the output template and nothing else:
Template:
[group 1]{item 1, item2, ...}
[group 2]{item 3, item 4, ...}
...
Items:
shoe.txt
tv.txt
fish.txt
bird.txt
apple.txt
dog.txt
lemon.txt
fish.pdf
apple.pdf
lol.dmg
MY_ANSWER "Here are the groups based on the file extensions:\n[group 1]{shoe.txt, tv.txt, fish.txt, bird.txt, apple.txt, dog.txt, lemon.txt}\n[group 2]{fish.pdf, apple.pdf}\n[group 3]{lol.dmg}"


'"Here are the groups based on the file extensions:\\n[group 1]{shoe.txt, tv.txt, fish.txt, bird.txt, apple.txt, dog.txt, lemon.txt}\\n[group 2]{fish.pdf, apple.pdf}\\n[group 3]{lol.dmg}"'

In [ ]:
file = open('dataTable3.json') #NOTE: hardCoded
fileData = json.load(file)
#fileDataS = json.dumps(fileData)

#create message to send to different agent
#NOTE: 1 layer simple sort, based on name
file_keys = fileData.keys()
message = ''
for k in file_keys:
    try:
        message =  message + '\n file: "' + k + '" content: "' + fileData[k]['summary'] + '"'
    except KeyError:
        continue
print(message)
    

In [ ]:
agentName = "interfaceAgent"

agent_1 = client.create_agent(
    name=agentName,
    tools=[x.name for x in client.list_tools()]
    )